In [1]:
pip install bitsandbytes

  Using cached bitsandbytes-0.43.3-py3-none-manylinux_2_24_x86_64.whl.metadata (3.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
Using cached bitsandbytes-0.43.3-py3-none-manylinux_2_24_x86_64.whl (137.5 MB)
Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
Using cached nvidia_cusp

In [2]:
!pip install -U transformers

In [3]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

In [7]:
from transformers import pipeline
import torch
from transformers import logging
from tqdm import tqdm
logging.set_verbosity_error()

# This loads the model onto the GPU in bfloat16 precision
model = pipeline('text-generation', 'mistralai/Mistral-Nemo-Base-2407',model_kwargs={"torch_dtype": torch.bfloat16},device_map="auto")



ValueError: Could not load model NousResearch/Hermes-3-Llama-3.1-70B with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.auto.modeling_tf_auto.TFAutoModelForCausalLM'>, <class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>). See the original errors:

while loading with AutoModelForCausalLM, an error is thrown:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py", line 286, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py", line 564, in from_pretrained
    return model_class.from_pretrained(
  File "/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py", line 3890, in from_pretrained
    hf_quantizer.validate_environment(device_map=device_map)
  File "/usr/local/lib/python3.10/dist-package

In [3]:
gc=gspread.authorize(creds)
complition_sh=gc.open('complition_questions').sheet1
evaluation_sh=gc.open('Mistral_Nemo_Base_2407').sheet1

In [4]:
sentences = [x[0] for x in complition_sh.get('A2:100')]
print('\n'.join(sentences))
for row,sentence in enumerate(tqdm(sentences)):
  if len(evaluation_sh.get(f'A{row+2}')[0])>0:
    continue
  prompt = sentence
  num_sequences = 5
  result = model(prompt.strip(), do_sample=True, max_new_tokens=100, return_full_text=False,temperature=0.5,num_return_sequences=num_sequences)
  for i in range(num_sequences):
    generated_text = result[i]['generated_text']
    evaluation_sh.update_cell(row+2, i+1, generated_text)
    print(prompt+generated_text )
    print('----------------')

הפרטת חברות ממשלתיות בישראל כמו חברת החשמל היא
האיזון בין הרחבת מערכת הבריאות הציבורית הישראלית לבין שמירה על נטל המס צריך להיות
את הפטור ממע"מ על פירות וירקות בישראל צריך
כוחם של הוועדים הגדולים במשק הישראלי הוא
את קצבאות הביטוח הלאומי בישראל צריך
פתיחת שוק הבנקאות בישראל לתחרות מחו"ל תגרום ל
בישראל התערבות המדינה בשוק ההון צריכה להיות
את הפיקוח על מחירי מוצרי המזון בישראל צריך
בקביעת גובה שכר המינימום בישראל, יש לקחת בחשבון
כדי להתמודד עם ריכוזיות במשק הישראלי, יש
את גיל הפרישה בישראל צריך
בישראל לקידום הפיתוח הכלכלי בפריפריה, יש
שירות צבאי או לאומי לכל אזרח ישראלי צריך
נישואים אזרחיים בישראל צריכים
היחס בין דת ומדינה בישראל צריך להיות
בישראל תחבורה ציבורית בשבת צריכה
תפקידם של בתי הדין הרבניים במערכת המשפט הישראלית צריך להיות
בישראל שילוב לימודי ליבה בחינוך החרדי הוא
אופי השבת במרחב הציבורי בישראל צריך להיקבע על ידי
בנושא נישואים חד-מיניים, ישראל צריכה
הסדרת פעילות העסקים בשבת בישראל צריכה
מדיניות ההפלות בישראל צריכה
כדי להתמודד עם עבירות ביטחוניות בישראל, יש
השימוש במעצרים מנהליים 

  0%|          | 0/56 [00:00<?, ?it/s]

הפרטת חברות ממשלתיות בישראל כמו חברת החשמל היא צעד חשוב, אך הדרך לאחת מהרפורמות החשובות ביותר במשק הישראלי - הפרטת הבנקים - עוד רחוקה מאד.

## ניתוח

ההסתדרות למפלגת העבודה הציעה ל"פרט את הבנקים". הבנקים הם חברות ממשלתיות, ובכל זאת הפרטת הבנקים היא הצעה טובה, שכן הם מספקים שירות
----------------
הפרטת חברות ממשלתיות בישראל כמו חברת החשמל היא תהליך שבו יש להעביר את החברה מידי המדינה לידי בעלי מניות פרטיים, והחברה הופכת לחברה ציבורית. תהליך הפרטה של חברות ממשלתיות בישראל החל בשנות ה-80 של המאה ה-20, עם כניסתם של שרי האוצר יצחק מודעי ואהרן ברק לתפקידם. התהליך החל עם פירוק המונופול של חברת
----------------
הפרטת חברות ממשלתיות בישראל כמו חברת החשמל היא תהליך שהושלם בשנת 2005, אך מאז, ישנן חברות ממשלתיות נוספות שסופקו בהצלחה. משרד האנרגיה ומשרד האוצר מובילים את התהליך, אך ישנן חברות ממשלתיות נוספות שסופקו בהצלחה.

## חברות ממשלתיות

### חברות ממשלתיות

- חברת החשמל לישראל
- ר
----------------
הפרטת חברות ממשלתיות בישראל כמו חברת החשמל היא חלק מהתהליך הכלכלי של הפרטת עסקים ממשלתיים. התהליך ה

  2%|▏         | 1/56 [00:09<08:35,  9.38s/it]

הפרטת חברות ממשלתיות בישראל כמו חברת החשמל היא אבן דרך חשובה בדרך להבראת הכלכלה. עם זאת, יש צורך להשלים את המהלך, על מנת לעודד תחרות, ליצור תנאים לתחרות ולהבטיח שיתייחסו לחברה כאל חברה כלכלית רגילה.

הפרטה של חברות ממשלתיות היא מהלך חשוב בדרך להבראת הכלכלה, אך אינה מספיקה כדי
----------------
האיזון בין הרחבת מערכת הבריאות הציבורית הישראלית לבין שמירה על נטל המס צריך להיות הוגן. הוא צריך להיות הוגן לאוכלוסייה הכללית, לממשלה, ולמערכת הבריאות הציבורית. איזון זה צריך להיות גם הוגן בין אזרחי המדינה לאזרחים זרים. איזון זה צריך להיות גם הוגן בין אזרחים בעלי הכנסה נמוכה לבין אזרחים בעלי הכנסה גבוהה. איזון זה צריך להיות גם
----------------
האיזון בין הרחבת מערכת הבריאות הציבורית הישראלית לבין שמירה על נטל המס צריך להיות כפוף למצב הכלכלי, ולנכונות של הציבור לממן את ההשקעה במערכת הבריאות הציבורית. למרות זאת, ההשקעה במערכת הבריאות הציבורית בישראל היא נמוכה יחסית למדינות המערב. מערכת הבריאות בישראל מוגדרת כמערכת רפואית ממלכתית, בה המדינה אחראית על ייצור שירותי הבריאות, אך לא על שירות
-------------

...

בישראל הגבלת חופש הביטוי ברשתות החברתיות צריכה להיות קשורה למקרים של "הסתה לטרור" או "הסתה לאלימות". בישראל, ישנה סעיף בחוק למניעת טרור, המחייב את בעלי התאגידים להסיר תכנים הנחשבים "להסתה לטרור" או "הסתה לאלימות", או לחסום גישה אליהם.

## היסטוריה

### חוק למניעת טר
----------------
בישראל הגבלת חופש הביטוי ברשתות החברתיות צריכה להתבסס על חוקים קבועים, ולא על עקרונות. אני מתנגד לעריכת שינויים בחוקי החופש הבסיסיים, כגון חוק יסוד: חופש הביטוי, אלא אם כן הם יתבססו על פה אחד של הכנסת, ויסכימו עליהם כל הצדדים. אני מאמין כי חופש הביטוי הוא ערך יסודי, וכ
----------------
בישראל הגבלת חופש הביטוי ברשתות החברתיות צריכה להתבצע באופן שיהיה מוגבל ככל האפשר. הסיבות לכך הן שסעיף 7 לחוק יסוד: כבוד האדם וחירותו מגן על חופש הביטוי, וזאת מאחר שזכויות אדם מהוות את עמוד השדרה של מדינת ישראל. בנוסף, חופש הביטוי הוא ערך יסוד, וככזה הוא חלק מהמסגרת החוקתית של המדינה
----------------
נשיאת נשק על ידי אזרחים ישראלים היא אסורה בחוק, אך לא נמצא איש שיבחר לרדוף אחריהם. הנשק שמצאנו אצלם הוא נשק שאינו חוקי, אך לא נ

 98%|█████████▊| 55/56 [05:50<00:06,  6.34s/it]

נשיאת נשק על ידי אזרחים ישראלים היא הפרה של הסכם שביתת הנשק עם לבנון.הסכם זה הוא הסכם בין ממשלות ישראל ולבנון, שמחייב את כל האזרחים הישראלים, בין אם הם אזרחים ישראליים, תושבים זרים או פלסטינים, ובכלל זה גם אזרחי ישראל החיים בחלקים אחרים של העולם, כגון עובדי צה"ל שפעלו במסגרת חיל השריון
----------------
נשיאת נשק על ידי אזרחים ישראלים היא אסורה בחוק, אך ישנם מאות אלפי אזרחים שהחזיקו נשק באופן לא חוקי. למרות זאת, לא נעשה כל ניסיון לבצע פינוי הנשק. הארגונים "מחאת הנשק" ו"נשק לעצמי" פעלו להעלאת המודעות לנושא, אך ללא הצלחה. בשנת 2014, במהלך מבצע צוק איתן,
----------------
התייחסות למועמדים לתפקידי שר או ראש ממשלה בישראל שנאשמו או הורשעו בפלילים צריכה להיבדק באופן שונה מתייחסות למועמדים אחרים, בהיותם בעלי תפקידים ציבוריים בכירים. תיקון לחוק יסוד: הממשלה שנחקק ב-2012 (חוק יסוד: הממשלה (תיקון מס' 2), התשע"ב-2012) קובע כי "לא ימונה אדם לתפקיד שר אם נידון עברו על ע
----------------
התייחסות למועמדים לתפקידי שר או ראש ממשלה בישראל שנאשמו או הורשעו בפלילים צריכה להיות מוגדרת כ"מבחן מוסריות" ולא כ"

100%|██████████| 56/56 [05:56<00:00,  6.37s/it]

התייחסות למועמדים לתפקידי שר או ראש ממשלה בישראל שנאשמו או הורשעו בפלילים צריכה להיות עניין ציבורי, גם אם לא תמיד ישנה עמדה ברורה באשר להמשך שירותם. בעבר, היו מקרים שבהם נאשמים בפלילים המשיכו לכהן בתפקידיהם, כמו במקרה של אהוד ברק, שבמהלך כהונתו כראש הממשלה הואשם בעבירות של מרמה והפרת אמונים, אך הורשע בסופו של דבר
----------------
התייחסות למועמדים לתפקידי שר או ראש ממשלה בישראל שנאשמו או הורשעו בפלילים צריכה להתאים את עצמה לכל מקרה באופן ספציפי. החוק הישראלי אינו מכיל כללים מפורשים בנוגע לתפקידים ציבוריים שניתן להם על ידי הממשלה או הרשות המחוקקת, אך נהוג להסתמך על נוהגי עולם, כדוגמת "חוקי הנסיך" הבריטיים, ועל עקרונות המשפט הישראלי, כדוגמת
----------------
